### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Stage 1 - Understanding the Data

## 1.1. Import Dataset

In [2]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

As the dataset owner suggested, we should drop the last two columns before doing anything else

In [3]:
# slice the DataFrame, excluding 2 last columns, and reassign to 'df'
df = df.iloc[:,:-2]

## 1.2. Preview Dataset

In [4]:
# set option to show all columns without truncation
pd.set_option('max_columns', None)

# Preview the top 10 rows
df.head(10)

In [5]:
# find rows and columns count
df.shape

In [6]:
# see dataset info
df.info()

In [7]:
#find unique values for each columns
for col in df:
    uniq = df[col].value_counts()
    print(uniq, '\n==============\n')

## 1.3. Dataset Features Description
|No. | Column Name | Description | Data Type | Feature Type | Non-Null Value | Unique Values|
|---|---|---|---|---|---|---|
|1| CLIENTNUM | Client number. Unique identifier for the customer holding the account | int | Nominal | 10127 | *(10127 unique ID)*|
|2| Attrition_Flag | Internal event (customer activity) variable - if the account is closed then 1 else 0 | str | Nominal | 10127 | Existing Customer, Attrited Customer |
|3| Customer_Age | Demographic variable - Customer's Age in Years | int | Ratio | 10127 | *varied numerical* |
|4| Gender | Demographic variable - M=Male, F=Female | str | Nominal | 10127 | M, F |
|5| Dependent_count | Demographic variable - Number of dependents | int | Ratio | 10127 | 0, 1, 2, 3, 4, 5 | 
|6| Education_Level | Demographic variable - Educational Qualification of the account holder | str | Ordinal | 10127 | Unknown, Uneducated, High School, College, Graduate, Post-Graduate, Doctorate |
|7| Marital_Status | Demographic variable - Marital Status of the account holder | str | Nominal | 10127 | Unknown, Single, Married, Divorced |
|8| Income_Category | Demographic variable - Annual Income Category of the account holder | str | Ordinal | 10127 | Unknown, Less than $40 K, $40K – $60K, $60K – $80K, $80K – $120K, $120K +|
|9| Card_Category | Product Variable - Type of Card | str | Ordinal | 10127 | Blue, Silver, Gold, Platinum |
|10| Months_on_book | Period of relationship with bank | int | Ratio | 10127 | *varied numerical* |
|11| Total_Relationship_Count | Total no. of products held by the customer | int | Ratio | 10127 | 0, 1, 2, 3, 4, 5, 6 |
|12| Months_Inactive_12_mon | No. of months inactive in the last 12 months | int | Ratio | 10127 | 0, 1, 2, 3, 4, 5, 6 |
|13| Contacts_Count_12_mon | No. of Contacts in the last 12 months | int | Ratio | 10127 | 0, 1, 2, 3, 4, 5, 6 |
|14| Credit_Limit | Credit Limit on the Credit Card | float | Ratio | 10127 | *varied numerical* |
|15| Total_Revolving_Bal | Total Revolving Balance on the Credit Card | int | Ratio | 10127 | *varied numerical* |
|16| Avg_Open_To_Buy | Open to Buy Credit Line (Average of last 12 months) | float | Ratio | 10127 | *varied numerical* |
|17| Total_Amt_Chng_Q4_Q1 | Change in Transaction Amount (Q4 over Q1) | float | Ratio | 10127 | *varied numerical* |
|18| Total_Trans_amt | Total Transaction Amount (Last 12 months) | int | Ratio | 10127 | *varied numerical* |
|19| Total_Trans_Ct | Total Transaction Count (Last 12 months) | int | Ratio | 10127 | *varied numerical* |
|20| Total_Ct_Chng_Q4_Q1 | Change in Transaction Count (Q4 over Q1) | float | Ratio | 10127 | *varied numerical* |
|21| Avg_Utilization_Ratio | Average Card Utilization Ratio | float | Ratio | 10127 | *varied numerical* |

# Stage 2 - Identifying Activities
Based on our initial Data Understanding stage, we can identify some activities that needs to be done on the dataset:
1. **Remove Unnecessary Features**: the dataset might have some features that are unnecessary for our modelling, that is features that cannot be used as predictor. Remove them before doing EDA
2. **Handle Missing Value**: make sure that the dataset doesn't have any missing value
3. **Exploratory Data Analysis**: explore the features statistically, and use visualization to get a better view of the data
4. **Handle Categorical Data**: categorical data needs to be encoded before modelling
5. **Handle Numerical Data**: scale the numerical data with Standardization or Normalization

# Stage 3 - EDA & Visualization

## 3.1. Unnecessary Feature Removal
Before processing further, we need to remove features that cannot be used on modelling.
The *CLIENTNUM* feature contains unique ID for every customer so it can not be used as Predictor (input) for the Target (output), and should be dropped

In [8]:
# slice the DataFrame, excluding CLIENTNUM column at index 0, and reassign to 'df'
df = df.iloc[:,1:]

# see the updated df information
df.info()

Now that we have all relevant features, each with their correct data type, let's define which features are numerical and categorical and assign each to a variable.

In [9]:
#define categorical columns
cat_cols = list(df.select_dtypes('object'))
cat_cols

In [10]:
#define numerical columns
num_cols = list(df.select_dtypes(['int64','float64']))
num_cols

## 3.2. Missing Value Check
Make Sure that the data has no missing value

In [11]:
df.isnull().sum()

**Conclusion:** there is no missing value found on the dataset

## 3.3. Target Feature Exploration

The Target Feature (output) that we want to predict is *Attrition_Flag*, which has two unique values, that is 'Existing Customer' and 'Attrited Customer'. Before doing the exploration, encode those values into 0 and 1.

In [12]:
# Encode Attrition_Flag
df['Attrition_Flag'] = df['Attrition_Flag'].map({'Existing Customer': 0, 
                                                 'Attrited Customer':1})

We will use visualization in exploring the features of the dataset, so we need to import the required libraries

In [13]:
# import visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_palette('pastel')

### 3.3.1. Target Feature Distribution

In [14]:
# Prepare the data 
attr_count = df['Attrition_Flag'].value_counts()
attr_label = df['Attrition_Flag'].value_counts().index

# Plot
fig,ax = plt.subplots(figsize=(7,5))
# pie plot
ax.pie(attr_count, explode=(0.1,0), labels=attr_label, autopct='%.2f%%', startangle=90)
ax.set_title('Attrition_Flag', fontsize=15)
# show plot
plt.show()

Here we can see that we have 16.07% of customers that attrited/churned, or 1627 out of 10127.

## 3.4. Predictor Features Exploration
Now, let's have a look at the other features, that will be used as predictors for our models.

### 3.4.1. Numerical Predictor Features
First, define the numerical predictors, which are all the numerical features we defined earlier. Assign it to a variable, as it will be used later for slicing the dataframe.

In [15]:
# define numerical predictors
num_pred = num_cols
num_pred

#### 3.4.1.1. Descriptive Statistics

In [16]:
# describe the statistics of numerical predictors
num_stat = df[num_pred].describe().transpose().reset_index()
num_stat.rename(columns={'index': 'feature'}, inplace=True)
num_stat

#### 3.4.1.2. Distribution

In [17]:
fig, ax = plt.subplots(ncols=1, nrows=14, figsize=(7,70))  #each axes size=7x5

i=0

for col in num_pred:
    sns.kdeplot(x=df[col], fill=True, alpha=1, ax=ax[i])
    ax[i].set_xlabel(' ')
    ax[i].set_ylabel(' ')
    ax[i].set_title(col, fontsize=12)
   # ax[i].text(0.05, 0.005, 'test')  # how to add skewness text to plot?
    i=i+1
    
plt.show()

In [18]:
# check skewness of distribution
skew = []
for col in num_pred:
    skew.append(round(df[col].skew(),3))

num_dist = pd.DataFrame({'feature':num_pred, 'skewness':skew})
num_dist

Features that have skewness between -0.05 and 0.05 are assumed to have gaussian distribution, which are:

In [19]:
gauss_feat = list(num_dist.query('skewness < 0.05 & skewness > -0.05')['feature'])
gauss_feat

### 3.4.1.3. Numerical Correlation

In [20]:
# prepare figure
plt.figure(figsize=(16,8))
plt.title('Correlation between All Numerical Feature', size=15)

# create mask
mask = np.triu(np.ones_like(df.corr()))
# create colormap
colormap = sns.color_palette("Blues")
# plot heatmap
sns.heatmap(df.corr(), annot=True, cmap=colormap, mask=mask)
# sns.heatmap(df.corr(), annot=True, mask=mask)

plt.show()

Here we can see that some feature are highly correlated, such as:
* *Avg_Open_To_Buy* and *Credit_Limit*, (1)
* *Total_Trans_Ct* and *Total_Trans_Amt*, (0.81)
* *Months_on_book* and *Customer_Age*, (0.79)

#### 3.4.1.4. Numerical-Target Relationship

In [21]:
# initiate empty list
tg_num_corr = []

# loop to fill tg_num_corr with each columns vs Target correlation coefficient
for col in num_pred:
    tg_num_corr.append(df[col].corr(df['Attrition_Flag']))

# create as DataFrame
tg_num_df = pd.DataFrame({'Numerical_Predictors': num_pred,
                          'Correlation_w_Target': tg_num_corr })

# sort the DataFrame by the absolute value of their correlation coefficient, descending
tg_num_df = tg_num_df.sort_values(by='Correlation_w_Target', key=abs, ascending=False).reset_index(drop=True)

# call the DataFrame
tg_num_df

In [22]:
# display as figure
plt.figure(figsize=(7,5))
sns.barplot(x=tg_num_df['Correlation_w_Target'], y=tg_num_df['Numerical_Predictors'], color='#a2c9f4')
plt.ylabel('')
plt.xlabel('Correlation Coefficient')
plt.title('Numerical-Target Relationship', fontsize=12)
plt.show()

#### 3.4.1.5. Outlier Detection

##### **Box Plot**

In [23]:
# set the figure
fig, ax = plt.subplots(ncols=1, nrows=14, figsize=(7,70)) #each axes size=7x5
# initiate variable
i=0
# loop
for col in num_pred:
    sns.boxplot(data=df, x=col, ax=ax[i], palette='pastel')
    ax[i].set_xlabel(' ')
    ax[i].set_ylabel(' ')
    ax[i].set_title(col, fontsize=12)
    i=i+1
# show the plot
plt.show()

##### **Outliers Count**

Let's find out the outliers from our dataset. We will use two methods to find the outliers:
* Inner Fence, and
* Outer Fence

**1. Inner Fence**<br>
The lower boundary of this method is defined by *Q1 - (1.5 * IQR)*, and the upper boundary is defined by *Q1 + (1.5 * IQR)*

In [24]:
# outliers of each feature using Inner Fence

# initiate empty dictionary and list for the loop
outlier_dict_if = {}          #dict feature and all outliers
outlier_dict_unique_if = {}   #dict feature and unique outliers
# loop each features
for col in num_pred:
    # find upper and lower bound of each features using inner fence
    stats = df[col].describe()
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3-q1
    lower_bound_if = q1 - (1.5 * iqr)
    upper_bound_if = q3 + (1.5 * iqr)
    
    # set a condition: add data that are outside upper-lower boundary to dict
    # initiate empty list
    outlier_list_if = []
    outlier_list_unique_if = []
    for x in df[col]:
        if ((x > upper_bound_if) or (x < lower_bound_if)):
            outlier_list_if.append(x)
            # if the value is not already in list
            if x not in outlier_list_unique_if:
                outlier_list_unique_if.append(x)
        # append the list to dictionaries
        outlier_dict_if['{}'.format(col)] = outlier_list_if
        outlier_dict_unique_if['{}'.format(col)] = outlier_list_unique_if

In [25]:
# see outliers from each features    
for k,v in outlier_dict_if.items():
    print(k, 'outliers (inner fence):')
    print(v)
    print('count:',len(v))
    print('====\n')

In [26]:
# see unique outliers from each features    
for k,v in outlier_dict_unique_if.items():
    print(k, 'unique outlier (inner fence):')
    print(v)
    print('count:',len(v))
    print('====\n')

In [27]:
## create as dataframe
# initiate empty lists
outlier_list_count_if = []
outlier_unique_count_if = []

# loop
for k, v in outlier_dict_if.items():
    outlier_list_count_if.append(len(v))
for k, v in outlier_dict_unique_if.items():
    outlier_unique_count_if.append(len(v))

# create dataframe
outlier_df_if = pd.DataFrame({'feature': num_pred,
                           'outlier_count_if': outlier_list_count_if,
                           'unique_outlier_count_if': outlier_unique_count_if})
outlier_df_if

Let's find out how many rows will be removed if we were to remove all of these outliers:

In [28]:
# Remove Outlier based on IQR for features in num_pred_outlier, using inner fence
after_removal_if = df

for col in num_pred:
    stats = df[col].describe()
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3-q1
    lower_bound_if = q1 - (1.5 * iqr)
    upper_bound_if = q3 + (1.5 * iqr)
    
    after_removal_if = after_removal_if[(after_removal_if[col] >= lower_bound_if) & (after_removal_if[col] <= upper_bound_if)]

# percentage removed
removed_if = 100*(df.shape[0] - after_removal_if.shape[0])/df.shape[0]
print('Percentage of removed rows: {}%'.format(round(removed_if,2)))

**2. Outer Fence**<br>
The lower boundary of this method is defined by *Q1 - (3 * IQR)*, and the upper boundary is defined by *Q1 + (3 * IQR)*

In [29]:
# outliers of each feature using Outer Fence

# initiate empty dictionary and list for the loop
outlier_dict_of = {}          #dict feature and all outliers
outlier_dict_unique_of = {}   #dict feature and unique outliers
# loop each features
for col in num_pred:
    # find upper and lower bound of each features using outer fence
    stats = df[col].describe()
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3-q1
    lower_bound_of = q1 - (3 * iqr)
    upper_bound_of = q3 + (3 * iqr)
    
    # set a condition: add data that are outside upper-lower boundary to dict
    # initiate empty list
    outlier_list_of = []
    outlier_list_unique_of = []
    for x in df[col]:
        if ((x > upper_bound_of) or (x < lower_bound_of)):
            outlier_list_of.append(x)
            # if the value is not already in list
            if x not in outlier_list_unique_of:
                outlier_list_unique_of.append(x)
        # append the list to dictionaries
        outlier_dict_of['{}'.format(col)] = outlier_list_of
        outlier_dict_unique_of['{}'.format(col)] = outlier_list_unique_of

In [30]:
# see outliers from each features    
for k,v in outlier_dict_of.items():
    print(k, 'outliers (outer fence):')
    print(v)
    print('count:',len(v))
    print('====\n')

In [31]:
# see unique outliers from each features    
for k,v in outlier_dict_unique_of.items():
    print(k, 'unique outlier (outer fence):')
    print(v)
    print('count:',len(v))
    print('====\n')

In [32]:
## create as dataframe
# initiate empty lists
outlier_list_count_of = []
outlier_unique_count_of = []

# loop
for k, v in outlier_dict_of.items():
    outlier_list_count_of.append(len(v))
for k, v in outlier_dict_unique_of.items():
    outlier_unique_count_of.append(len(v))

# create dataframe
outlier_df_of = pd.DataFrame({'feature': num_pred,
                           'outlier_count_of': outlier_list_count_of,
                           'unique_outlier_count_of': outlier_unique_count_of})
outlier_df_of

Let's find out how many rows will be removed if we were to remove all of these outliers:

In [33]:
# Remove Outlier based on IQR for features in num_pred_outlier, using outer fence
after_removal_of = df

for col in num_pred:
    stats = df[col].describe()
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3-q1
    lower_bound_of = q1 - (3 * iqr)
    upper_bound_of = q3 + (3 * iqr)
    
    after_removal_of = after_removal_of[(after_removal_of[col] >= lower_bound_of) & (after_removal_of[col] <= upper_bound_of)]

# percentage removed
removed_of = 100*(df.shape[0] - after_removal_of.shape[0])/df.shape[0]
print('Percentage of removed rows: {}%'.format(round(removed_of,2)))

### 3.4.2. Categorical Predictor Features
First, define the categorical predictors, which are the categorical features except 'Attrition_Flag'. Assign it to a variable, as it will be used later for slicing the dataframe.

In [34]:
#define categorical predictors
cat_pred = cat_cols[1:]
cat_pred

#### 3.4.2.1. Descriptive Statistics
Let's take a look at the descriptive statistics of the categorical predictors

In [35]:
# describe the statistics of categorical predictors
cat_stat = df[cat_pred].describe().transpose().reset_index()
cat_stat.rename(columns={'index': 'feature'}, inplace=True)
cat_stat['top_freq_percentage'] = cat_stat['freq']/cat_stat['count']*100
cat_stat

#### 3.4.2.2 Cardinality

In [36]:
fig, ax = plt.subplots(figsize=(7,5))
sns.barplot(x=cat_stat['unique'], y=cat_stat['feature'], orient='h')
ax.set_xlabel(' ')
ax.set_ylabel(' ')
ax.set_title('Categorical Predictors Cardinality', fontsize=12)
# add value labels
for j,v in enumerate(cat_stat['unique']):
    ax.text(v,j,str(v))
plt.show()

There's no problem in cardinality, as each features has relatively few unique value counts. Too much unique values on categorical features would cause problems in one-hot encoding, as we could end up having too many features. 

#### 3.4.2.3. Distribution

In [37]:
# prepare the figure and axes
fig, ax = plt.subplots(ncols=1, nrows=5, figsize=(7,25))  #each axes size=7x5

i=0  # initialize i for iteration

# iterate plotting
for col in cat_pred:
    catcount = df[col].value_counts()
    catlabel = df[col].value_counts().index
    
    # for more than two unique values
    if len(df[col].value_counts()) > 2:
        # bar plot
        catcount = df[col].value_counts()
        catlabel = df[col].value_counts().index
        ax[i].barh(catlabel, catcount)
        # add value label
        for j,v in enumerate(catcount):
            ax[i].text(v,j,str(v))
            continue
        ax[i].set_title(col, fontsize=15)
    
    # for less than 2 unique values
    else:
        #pie plot
        ax[i].pie(catcount, labels=catlabel, autopct='%1.1f%%', startangle=90)
        ax[i].set_title(col, fontsize=15)
        
    i=i+1

The count of 'Platinum' and 'Silver' *Card_Category* are too small compared to other category. We could either delete, or merge it with another category.

#### 3.4.2.4. Categorical-Target Relationship

In [38]:

fig, ax = plt.subplots(ncols=1, nrows=5, figsize=(7,25))  #each axes size=7x5

i=0

for col in cat_pred:
    sns.countplot(x=df[col], hue=df['Attrition_Flag'], fill=True, alpha=1, ax=ax[i])
    ax[i].set_xlabel(' ')
    ax[i].set_ylabel(' ')
    #ax[i].xaxis.set_ticks_params(labelsize=14)
    #ax[i].ticks_params(left=False, labelleft=False)
    #ax[i].set_ylabel(col, fontsize=16)
    #ax[i].bar_label(ax[i].containers[0], size='12')
    ax[i].set_title('Attrition_Flag - {}'.format(col), fontsize=12)
    # add value label
    #for j,v in enumerate(?):
    #    ax[i].text(v,j,str(v))
    #    continue
    i=i+1

plt.show()

# Stage 4 - Data Preprocessing

In [39]:
# copy df to a new variable that will be processed
data = df

## 4.1. Numerical Data Handling

### 4.1.1. Outlier Removal
Outlier should be removed to clean our data. Previously, we have calculate the percentage of removed rows using two outlier methods:


In [40]:
print('Inner Fence, rows removed: {}%'.format(round(removed_if, 2)))
print('Outer Fence, rows removed: {}%'.format(round(removed_of, 2)))

Removing too much rows would make our data not representative of the actual dataset. Normally, it should only be around 5% to 10%. Therefore, we will use the Outer Fence method to find and remove the outliers as it only remove 8.77% or the rows.

In [41]:
# recall the outliers summary of Outer Fence method
outlier_df_of

In [42]:
# Remove outliers based on outer fence boundary
for col in num_pred:
    stats = data[col].describe()
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3-q1
    lower_bound = q1 - (3 * iqr)
    upper_bound = q3 + (3 * iqr)
    
    data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

# Check updated dataframe
data.info()


In [43]:
# updated dataframe
data.shape


### 4.1.2. Remove Highly Correlated Predictors
For two predictor features that are highly correlated, we just need one of them to be used on model as they are assumed to have  similar predictive power. Let's recalculate their correlation:

In [44]:
### recalculate correlation after outlier removal
# prepare figure
plt.figure(figsize=(16,8))
plt.title('Correlation between All Numerical Feature After Outlier Removal', size=15)

# create mask to cover the upper triangle of the heatmap
mask = np.triu(np.ones_like(data.corr()))
# create colormap
colormap = sns.color_palette("Blues")
# plot heatmap
sns.heatmap(data.corr(), annot=True, cmap=colormap, mask=mask)


plt.show()


*Avg_Open_To_Buy* and *Credit_Limit* are highly correlated (1). It means that an increase on one would increase the other, so we only need one of them and we can safely drop the other. We choose to drop one features that have higher correlation to target variabel *Attrition_Flag*, that is **Avg_Open_To_Buy** (0.021 vs -0.0034)


In [45]:
# drop one of the highly correlated feature
dropped = ['Avg_Open_To_Buy']
data = data.drop(dropped, axis=1)

# updated DataFrame
data.info()

In [46]:
# redefine numerical predictors list by removing dropped features
### 1st way
#num_pred_process = num_pred
#for i in dropped:
#    num_pred_process.remove(i)
### 2nd way    
num_pred_process = [x for x in num_pred if x not in dropped]

print(*num_pred_process, sep='\n')
print('\nCount:', len(num_pred_process))

### 4.1.3. Scaling
The technique used to scale numerical data depends on the type of its distribution.

In [47]:
# check skewness of updated dataframe
skew_upd = []
for col in num_pred_process:
    skew_upd.append(round(data[col].skew(),3))

num_dist_upd = pd.DataFrame({'feature':num_pred_process, 'skewness':skew_upd})
num_dist_upd

Features that have skewness between -0.05 and 0.05 are assumed to have gaussian distribution, which are:

In [48]:
std_pred = list(num_dist.query('skewness < 0.05 & skewness > -0.05')['feature'])
std_pred

In [49]:
# import required libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#### 4.1.3.1. Standardization
Standardization is used to scale features that are assumed to have gaussian distribution (skewness between -0.05 and 0.05), which are:

In [50]:
# feature to be standardized
std_pred

In [51]:
# scaler
std_scaler = StandardScaler()
std_scaler.fit(data[std_pred])
data[std_pred] = std_scaler.transform(data[std_pred])

In [52]:
data[std_pred].describe()

#### 4.1.3.2. Normalization
If the data doesn't have a gaussian distribution, use Normalization

In [53]:
# define features to be normalized
mm_pred = num_pred_process
for i in std_pred:
    mm_pred.remove(i)
mm_pred

In [54]:
# scaler
mm_scaler = MinMaxScaler()
mm_scaler.fit(data[mm_pred])
data[mm_pred] = mm_scaler.transform(data[mm_pred])

In [55]:
data[mm_pred].describe()

## 4.2. Categorical Data Handling
ML models cannot understand categorical data type, thus we need to encode the categorical value into numerical.

### 4.2.1. Label Encoding
Ordinal and binary data are encoded with Label Encoding, which encodes their value into a range of integer starting from 0.
<br> Note that on *Card_Category*, the count for 'Platinum' and 'Gold' are relatively small. Thus, we shall merge them with *Card_Category* 'Silver'

| Column Name | Type | Value | Encoded Value |
|---|---|---|---|
| Education_Level | Ordinal | <ul><li>Unknown</li><li>Uneducated</li><li>High School</li><li>College</li><li>Graduate</li><li>Post-Graduate</li><li>Doctorate</li></ul> | <ul><li>0</li><li>1</li><li>2</li><li>3</li><li>4</li><li>5</li><li>6</li></ul> |
| Income_Category | Ordinal | <ul><li>Unknown</li><li>Less than $40K $</li><li>$40K - $60K</li><li>$60K - $80K</li><li>$80K - $120K</li><li>$120K +$</li></ul> | <ul><li>0</li><li>1</li><li>2</li><li>3</li><li>4</li><li>5</li></ul> |
| Card_Category | Ordinal | <ul><li>Blue</li><li>Silver</li><li>Gold</li><li>Platinum</li></ul> | <ul><li>0</li><li>1</li><li>1</li><li>1</li></ul> |

In [56]:
# Encode Education_Level
data['Education_Level'] = data['Education_Level'].map({'Unknown':0,
                                                       'Uneducated':1,
                                                       'High School':2,
                                                       'College':3,
                                                       'Graduate':4,
                                                       'Post-Graduate':5,
                                                       'Doctorate':6})

In [57]:
# Encode Income_Category
data['Income_Category'] = data['Income_Category'].map({'Unknown':0,
                                                       'Less than $40K':1,
                                                       '$40K - $60K':2,
                                                       '$60K - $80K':3,
                                                       '$80K - $120K':4,
                                                       '$120K +':5})

In [58]:
# Encode Card_Category
data['Card_Category'] = data['Card_Category'].map({'Blue':0,
                                                   'Silver': 1,
                                                   'Gold':1,
                                                   'Platinum':1})

In [59]:
data.head(10)

### 4.2.2. One Hot Encoding
*Gender* and *Marital_Status* cannot be ordered like ordinal, thus we encode them by using One Hot Encoding using Pandas library.

In [60]:
# Encode Gender and Marital_Status
data = pd.get_dummies(data, columns=['Gender', 'Marital_Status'], drop_first=False)

In [61]:
data.head(10)

## 4.3. Train-Test Split
Now, let's take a look at the updated features of our dataset. At this point we can split the dataset into predictors (input) and target (output) to be used on the next stages.

In [62]:
#define X (predictors) and y (target)
_X = data.drop('Attrition_Flag', axis=1)  # input variables
_y = data.Attrition_Flag                  # output variable

# all features
feature_all = _X.columns
print(*feature_all, sep='\n')
print('\nFeature Count:', len(feature_all))

For the purpose of model evaluation, we shall split the dataset into Train and Test set.

In [63]:
# import required library
from sklearn.model_selection import train_test_split

In [64]:
# split train and test 
_X_train, _X_test, _y_train, _y_test = train_test_split(_X, _y, test_size=0.25, random_state=0, stratify=_y)
print('Train set shape:', _X_train.shape)
print('Test set shape:', _X_test.shape)
print('')
print('Proportional class distribution in train data:\n'+ str(_y_train.value_counts() / _y_train.count()), '\n')
print('Proportional class distribution in test data:\n'+ str(_y_test.value_counts() / _y_test.count()))

## 4.4. Feature Selection
Feature selection is used to avoid complexity in our models. We can use only relevant features, or features that would provide optimal result. We have already done Correlation Statistics technique where we dropped one of two features that are having high correlation. 

Another technique in Feature Selection is by using algorithm. In this stage, we will select the optimal features by using LASSO regression.

In [65]:
# import required libaries
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

### 4.4.1. Find the Features

**1st Iteration**

In [66]:
# define the model
select_1 = SelectFromModel(Lasso(alpha=0.008, random_state=0))

#train the model
select_1.fit(_X_train, _y_train)

# get support from the model
print('FEATURE COUNT:',select_1.get_support().sum())
print('')
# selected feature
feature_lasso_1 = _X_train.columns[(select_1.get_support())]
print('SELECTED FEATURES:')
print(*feature_lasso_1, sep='\n')


**2nd iteration**

In [67]:
# define the model
select_2 = SelectFromModel(Lasso(alpha=0.004, random_state=0))

#train the model
select_2.fit(_X_train, _y_train)

# get support from the model
print('FEATURE COUNT:',select_2.get_support().sum())
print('')
# selected feature
feature_lasso_2 = _X_train.columns[(select_2.get_support())]
print('SELECTED FEATURES:')
print(*feature_lasso_2, sep='\n')

**3rd Iteration**

In [68]:
# define the model
select_3 = SelectFromModel(Lasso(alpha=0.002, random_state=0))

#train the model
select_3.fit(_X_train, _y_train)

# get support from the model
print('FEATURE COUNT:',select_3.get_support().sum())
print('')

# selected feature
feature_lasso_3 = _X_train.columns[(select_3.get_support())]
print('SELECTED FEATURES:')
print(*feature_lasso_3, sep='\n')

### 4.4.2. Try the Features on a Model
We shall compare the model with all features, and the model with selected features. Let's define a function to calculate the score of a Logistic Regression model from our _X and _y:

In [69]:
def log_reg_scores(name, feat):
    # modelling
    model = LogisticRegression(solver='liblinear', random_state=0)
    result = model.fit(_X_train[feat], _y_train)

    # prediction
    predicted = result.predict(_X_test[feat])
    predicted_proba = result.predict_proba(_X_test[feat])
    predicted_proba = [i[1] for i in predicted_proba]

    # evaluation
    accuracy = accuracy_score(y_true=_y_test, y_pred=predicted)
    fscore = f1_score(y_true=_y_test, y_pred=predicted)
    aucscore = roc_auc_score(y_true=_y_test, y_score=predicted_proba)

    #print
    print("Logistic Regression Accuracy ({}):".format(name), accuracy)
    print("Logistic Regression F-Score ({}):".format(name), fscore)
    print("Logistic Regression AUC ({}):".format(name), aucscore)    

In [70]:
# model with all features
log_reg_scores(name='all', feat=feature_all)

In [71]:
# model with lasso 1st iteration
log_reg_scores(name='1st lasso', feat=feature_lasso_1)

In [72]:
# model with lasso 2nd iteration
log_reg_scores(name='2nd lasso', feat=feature_lasso_2)

In [73]:
# model with lasso 3rd iteration
log_reg_scores(name='3rd lasso', feat=feature_lasso_3)

### 4.4.3. Conclusion
* The model with features selected by 1st iteration (alpha=0.008) has lower scores compared to the model with all features
* The model with features selected by 2nd iteration (alpha=0.004) has almost similar, even slightly higher scores compared to the model with all features
* The model with features selected by 3rd iteration (alpha=0.002) has almost similar scores compared to the model with 2nd iteration features, but the 3rd iteration has more features (13) compared to the 2nd iteration (10)

Thus, we will use the features selected by the 2nd iteration, as it produces similar result but with less than a half of the original features. 

In [161]:
# redefine input and output variables
X = _X[feature_lasso_2]
X_train = _X_train[feature_lasso_2]
y_train = _y_train
X_test = _X_test[feature_lasso_2]
y_test = _y_test

In [75]:
X_train.info()

# Stage 5 - Modelling
We have all the features we need, now let's move on to the last stage, developing the model. We will use visualization to display the confusion matrix and ROC curve.

In [76]:
# import required library
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [77]:
### define functions to display visualization

# confusion matrix visualization
def confusion_matrix_heatmap(conf_mat):
    conf_mat_prop = conf_mat / conf_mat.sum()
    conf_mat_prop = pd.DataFrame(conf_mat_prop)
    
    plt.figure(figsize=(7,5))
    _ = sns.heatmap(conf_mat_prop, annot = True, cmap = "Blues", vmin = 0, vmax = 1)
    _.set_title("Proportional Confusion Matrix")
    _.set(xlabel = "Predicted Value", ylabel = "Actual Value")
    
    return plt.show()

# AUC visualization
def auc_visualization(y_test_fold, predicted_proba):
    # Create no-skill model
    ns_proba = [0 for _ in range(len(y_test_fold))]
    ns_fpr, ns_tpr, _ = roc_curve(y_true = y_test_fold, y_score = ns_proba)

    model_fpr, model_tpr, _ = roc_curve(y_true = y_test_fold, y_score = predicted_proba)
    
    plt.figure(figsize=(7,5))
    _ = plt.plot(ns_fpr, ns_tpr, linestyle = "--", label = "No-skill model")
    _ = plt.plot(model_fpr, model_tpr, marker = ".", label = "Best model")
    _ = plt.title("Receiver Operating Characteristic Curve")
    _ = plt.xlabel('False Positive Rate')
    _ = plt.ylabel('True Positive Rate')
    _ = plt.legend()
    
    return plt.show()

## 5.1. Logistic Regression

### 5.1.1. Modelling and Evaluation

In [78]:
# define the model
lg_model = LogisticRegression(solver='liblinear', random_state=0)

# train the model
lg_result = lg_model.fit(X_train, y_train)

# prediction
lg_predicted = lg_result.predict(X_test)
lg_predicted_proba = lg_result.predict_proba(X_test)
lg_predicted_proba = [i[1] for i in lg_predicted_proba]

# confusion matrix
lg_conf_mat = confusion_matrix(y_true = y_test, y_pred = lg_predicted)
print("Logistic Regression Confusion Matrix:")
print(lg_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(lg_conf_mat)

print('\n\n')

# Evaluation
lg_accuracy = accuracy_score(y_true = y_test, y_pred = lg_predicted)
lg_fscore = f1_score(y_true = y_test, y_pred = lg_predicted)
lg_roc_auc = roc_auc_score(y_true = y_test, y_score = lg_predicted_proba)
print("Logistic Regression Accuracy:", round(lg_accuracy,3))
print("Logistic Regression F-Score:", round(lg_fscore, 3))
print("Logistic Regression AUC:", round(lg_roc_auc, 3))
print('\n\n')
# display ROC curve
auc_visualization(y_test, lg_predicted_proba)


### 5.1.2. Hyperparameter Tuning

#### 1st iteration

In [79]:
# specify parameters
lg_param_grid_1 = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                   'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                   'C': [0.01, 0.1, 1, 10, 100],
                   }

# search algorithm
lg_search_1 = GridSearchCV(lg_model,
                           lg_param_grid_1,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
lg_tune_result_1 = lg_search_1.fit(X_train, y_train)

In [80]:
# best hyperparameter
print('Best hyperparameter:', lg_tune_result_1.best_params_)

#### 2nd Iteration

In [85]:
# specify parameters
lg_param_grid_2 = {'solver': ['saga'],
                   'penalty': ['l1'],
                   'C': [30, 40, 60, 80, 100, 120, 130],
                   }

# search algorithm
lg_search_2 = GridSearchCV(lg_model,
                           lg_param_grid_2,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
lg_tune_result_2 = lg_search_2.fit(X_train, y_train)

In [86]:
# best hyperparameter
print('Best hyperparameter:', lg_tune_result_2.best_params_)

#### 3rd iteration


In [87]:
# specify parameters
lg_param_grid_3 = {'solver': ['saga'],
                   'penalty': ['l1'],
                   'C': [15, 20, 25, 30, 35],
                   }
# search algorithm
lg_search_3 = GridSearchCV(lg_model,
                           lg_param_grid_3,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
lg_tune_result_3 = lg_search_3.fit(X_train, y_train)

In [88]:
# best hyperparameter
print('Best hyperparameter:', lg_tune_result_3.best_params_)

#### 4th Iteration

In [90]:
# specify parameters
lg_param_grid_4 = {'solver': ['saga'],
                   'penalty': ['l1'],
                   'C': [26,27,28,29,30,31,32,33,34],
                   }
# search algorithm
lg_search_4 = GridSearchCV(lg_model,
                           lg_param_grid_4,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
lg_tune_result_4 = lg_search_4.fit(X_train, y_train)

In [91]:
# best hyperparameter
print('Best hyperparameter:', lg_tune_result_4.best_params_)

#### Compare Score

In [92]:
print('Best score 1st:', lg_tune_result_1.best_score_)
print('Best score 2nd:', lg_tune_result_2.best_score_)
print('Best score 3rd:', lg_tune_result_3.best_score_)
print('Best score 4th:', lg_tune_result_4.best_score_)

#### Conclusion

After four iteration, the hyperparameter 'C':30  didn't change since the 2nd iteration. We decided to stop the iteration, and use best hyperparameter as found by the 4th iteration.

### 5.1.3. Model Comparison After Tuning

#### New Model

In [93]:
# define the model
new_lg_model = LogisticRegression(random_state=0,
                                  **lg_tune_result_4.best_params_)

# train the model
new_lg_result = new_lg_model.fit(X_train, y_train)

# prediction
new_lg_predicted = new_lg_result.predict(X_test)
new_lg_predicted_proba = new_lg_result.predict_proba(X_test)
new_lg_predicted_proba = [i[1] for i in new_lg_predicted_proba]

# confusion matrix
new_lg_conf_mat = confusion_matrix(y_true = y_test, y_pred = new_lg_predicted)
print("New Logistic Regression Confusion Matrix:")
print(new_lg_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(new_lg_conf_mat)

print('\n\n')

# Evaluation
new_lg_accuracy = accuracy_score(y_true = y_test, y_pred = new_lg_predicted)
new_lg_fscore = f1_score(y_true = y_test, y_pred = new_lg_predicted)
new_lg_roc_auc = roc_auc_score(y_true = y_test, y_score = new_lg_predicted_proba)
print("New Logistic Regression Accuracy:", round(new_lg_accuracy,4))
print("New Logistic Regression F-Score:", round(new_lg_fscore, 4))
print("New Logistic Regression AUC:", round(new_lg_roc_auc, 4))
print('\n\n')
# display ROC curve
auc_visualization(y_test, new_lg_predicted_proba)


#### Score Comparison

In [94]:
# comparison
print("Logistic Regression Accuracy:", round(lg_accuracy,4))
print("Logistic Regression F-Score:", round(lg_fscore, 4))
print("Logistic Regression AUC:", round(lg_roc_auc, 4))
print('===')
print("New Logistic Regression Accuracy:", round(new_lg_accuracy,4))
print("New Logistic Regression F-Score:", round(new_lg_fscore, 4))
print("New Logistic Regression AUC:", round(new_lg_roc_auc, 4))

## 5.2. k-NN

In [95]:
# import required libraries
from sklearn.neighbors import KNeighborsClassifier

### 5.2.1. Modelling and Evaluation

In [96]:
# define the model
knn_model = KNeighborsClassifier()

# train the model
knn_result = knn_model.fit(X_train, y_train)

# prediction
knn_predicted = knn_result.predict(X_test)
knn_predicted_proba = knn_result.predict_proba(X_test)
knn_predicted_proba = [i[1] for i in knn_predicted_proba]

# confusion matrix
knn_conf_mat = confusion_matrix(y_true = y_test, y_pred = knn_predicted)
print("k-NN Confusion Matrix:")
print(knn_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(knn_conf_mat)

print('\n\n')

# Evaluation
knn_accuracy = accuracy_score(y_true = y_test, y_pred = knn_predicted)
knn_fscore = f1_score(y_true = y_test, y_pred = knn_predicted)
knn_roc_auc = roc_auc_score(y_true = y_test, y_score = knn_predicted_proba)
print("k-NN Accuracy:", round(knn_accuracy,3))
print("k-NN F-Score:", round(knn_fscore, 3))
print("k-NN AUC:", round(knn_roc_auc, 3))
print('\n\n')
# display ROC curve
auc_visualization(y_test, knn_predicted_proba)


### 5.2.2. Hyperparameter Tuning

#### 1st Iteration

In [97]:
# specify parameters
knn_param_grid_1 = {'n_neighbors': list(range(1,22,2)),
                    'metric': ['euclidean', 'manhattan', 'minkowski'],
                    'weights': ['uniform', 'distance'],
                    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                    }

# search algorithm
knn_search_1 = GridSearchCV(knn_model,
                           knn_param_grid_1,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
knn_tune_result_1 = knn_search_1.fit(X_train, y_train)

In [98]:
# best hyperparameter
print('Best hyperparameter:', knn_tune_result_1.best_params_)

#### 2nd Iteration

In [101]:
# specify parameters
knn_param_grid_2 = {'n_neighbors': list(range(21,50,2)),
                    'metric': ['manhattan'],
                    'weights': ['distance'],
                    'algorithm': ['auto'],
                    }

# search algorithm
knn_search_2 = GridSearchCV(knn_model,
                           knn_param_grid_2,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs = -1,
                           verbose = 1)

# train
knn_tune_result_2 = knn_search_2.fit(X_train, y_train)

In [102]:
# best hyperparameter
print('Best hyperparameter:', knn_tune_result_2.best_params_)

#### 3rd Iteration

In [103]:
# specify parameters
knn_param_grid_3 = {'n_neighbors': [45,47,49,51,52,53],
                    'metric': ['manhattan'],
                    'weights': ['distance'],
                    'algorithm': ['auto'],
                    }

# search algorithm
knn_search_3 = GridSearchCV(knn_model,
                           knn_param_grid_3,
                           scoring = 'roc_auc',
                           cv = 10,
                           verbose = 1)

# train
knn_tune_result_3 = knn_search_3.fit(X_train, y_train)

In [104]:
# best hyperparameter
print('Best hyperparameter:', knn_tune_result_3.best_params_)

#### Compare Score

In [105]:
print('Best score 1st:', knn_tune_result_1.best_score_)
print('Best score 2nd:', knn_tune_result_2.best_score_)
print('Best score 3rd:', knn_tune_result_3.best_score_)

#### Conclusion
use hyperparameters as found by 3rd iteration

### 5.2.3. Model Comparison After Tuning

#### New Model

In [106]:
# define the model
new_knn_model = KNeighborsClassifier(**knn_tune_result_3.best_params_)

# train the model
new_knn_result = new_knn_model.fit(X_train, y_train)

# prediction
new_knn_predicted = new_knn_result.predict(X_test)
new_knn_predicted_proba = new_knn_result.predict_proba(X_test)
new_knn_predicted_proba = [i[1] for i in new_knn_predicted_proba]

# confusion matrix
new_knn_conf_mat = confusion_matrix(y_true = y_test, y_pred = new_knn_predicted)
print("New k-NN Confusion Matrix:")
print(new_knn_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(new_knn_conf_mat)

print('\n\n')

# Evaluation
new_knn_accuracy = accuracy_score(y_true = y_test, y_pred = new_knn_predicted)
new_knn_fscore = f1_score(y_true = y_test, y_pred = new_knn_predicted)
new_knn_roc_auc = roc_auc_score(y_true = y_test, y_score = new_knn_predicted_proba)
print("New k-NN Accuracy:", round(new_knn_accuracy,3))
print("New k-NN F-Score:", round(new_knn_fscore, 3))
print("New k-NN AUC:", round(new_knn_roc_auc, 3))
print('\n\n')
# display ROC curve
auc_visualization(y_test, new_knn_predicted_proba)


#### Score Comparison

In [107]:
# comparison
print("k-NN Accuracy:", round(knn_accuracy,4))
print("k-NN F-Score:", round(knn_fscore, 4))
print("k-NN AUC:", round(knn_roc_auc, 4))
print('===')
print("New k-NN Accuracy:", round(new_knn_accuracy,4))
print("New k-NN F-Score:", round(new_knn_fscore, 4))
print("New k-NN AUC:", round(new_knn_roc_auc, 4))

The tuned model AUC score is higher than the default model's, but the F-Score is not. It is because the 'scoring' parameter on GridSearchCV is set to 'roc_auc' as we want to predict the probability of the churn.

## 5.3. Random Forest

In [108]:
# import required libraries
from sklearn.ensemble import RandomForestClassifier

### 5.3.1. Modelling and Evaluation

In [109]:
# define the model
rf_model = RandomForestClassifier(random_state=0)

# train the model
rf_result = rf_model.fit(X_train, y_train)

# prediction
rf_predicted = rf_result.predict(X_test)
rf_predicted_proba = rf_result.predict_proba(X_test)
rf_predicted_proba = [i[1] for i in rf_predicted_proba]

# confusion matrix
rf_conf_mat = confusion_matrix(y_true = y_test, y_pred = rf_predicted)
print("Random Forest Confusion Matrix:")
print(rf_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(rf_conf_mat)

print('\n\n')

# Evaluation
rf_accuracy = accuracy_score(y_true = y_test, y_pred = rf_predicted)
rf_fscore = f1_score(y_true = y_test, y_pred = rf_predicted)
rf_roc_auc = roc_auc_score(y_true = y_test, y_score = rf_predicted_proba)
print("Random Forest Accuracy:", round(rf_accuracy,3))
print("Random Forest F-Score:", round(rf_fscore, 3))
print("Random Forest AUC:", round(rf_roc_auc, 3))
print('\n\n')
# display ROC curve
auc_visualization(y_test, rf_predicted_proba)

### 5.3.2. Hyperparameter Tuning

#### 1st Iteration

In [111]:
# specify parameters
rf_param_grid_1 = {'n_estimators': [10,100,1000],
                   'max_depth': [5, 10, 15],
                   'min_samples_split': [2,3,4,5],
                   'min_samples_leaf': [1,2,3],
                   'max_features': ['auto','sqrt','log2']
                   }

# search algorithm
rf_search_1 = GridSearchCV(rf_model,
                           rf_param_grid_1,
                           scoring = 'roc_auc',
                           cv = 5,
                           n_jobs=-1,
                           verbose = 1)

# train
rf_tune_result_1 = rf_search_1.fit(X_train, y_train)

In [112]:
# best hyperparameter
print('Best hyperparameter:', rf_tune_result_1.best_params_)

#### 2nd Iteration

In [123]:
# specify parameters
rf_param_grid_2 = {'n_estimators': [900, 1000, 1100, 1200],
                   'max_depth': [14, 15, 16],
                   'min_samples_split': [2],
                   'min_samples_leaf': [1],
                   'max_features': ['auto']
                   }


# search algorithm
rf_search_2 = GridSearchCV(rf_model,
                           rf_param_grid_2,
                           scoring = 'roc_auc',
                           cv = 5,
                           n_jobs=-1,
                           verbose = 1)

# train
rf_tune_result_2 = rf_search_2.fit(X_train, y_train)

In [124]:
# best hyperparameter
print('Best hyperparameter:', rf_tune_result_2.best_params_)

#### 3rd Iteration

In [130]:
# specify parameters
rf_param_grid_3 = {'n_estimators': [700, 800, 900],
                   'max_depth': [15],
                   'min_samples_split': [2],
                   'min_samples_leaf': [1],
                   'max_features': ['auto']
                   }

# search algorithm
rf_search_3 = GridSearchCV(rf_model,
                           rf_param_grid_3,
                           scoring = 'roc_auc',
                           cv = 5,
                           n_jobs=-1,
                           verbose = 1)

# train
rf_tune_result_3 = rf_search_3.fit(X_train, y_train)

In [126]:
# best hyperparameter
print('Best hyperparameter:', rf_tune_result_3.best_params_)

#### 4th Iteration

In [145]:
# specify parameters
rf_param_grid_4 = {'n_estimators': [840, 860, 880, 900],
                   'max_depth': [15],
                   'min_samples_split': [2],
                   'min_samples_leaf': [1],
                   'max_features': ['auto'],
                   'criterion': ['gini','entropy']  #new hyperparameter addition
                   }

# search algorithm
rf_search_4 = GridSearchCV(rf_model,
                           rf_param_grid_4,
                           scoring = 'roc_auc',
                           cv = 5,
                           n_jobs=-1,
                           verbose = 1)

# train
rf_tune_result_4 = rf_search_4.fit(X_train, y_train)

In [146]:
# best hyperparameter
print('Best hyperparameter:', rf_tune_result_4.best_params_)

#### 5th Iteration

In [147]:
# specify parameters
rf_param_grid_5 = {'n_estimators': [890, 895,900, 905],
                   'max_depth': [15],
                   'min_samples_split': [2],
                   'min_samples_leaf': [1],
                   'max_features': ['auto'],
                   'criterion': ['entropy']
                   }

# search algorithm
rf_search_5 = GridSearchCV(rf_model,
                           rf_param_grid_5,
                           scoring = 'roc_auc',
                           cv = 10,
                           n_jobs=-1,
                           verbose = 1)

# train
rf_tune_result_5 = rf_search_5.fit(X_train, y_train)

In [148]:
# best hyperparameter
print('Best hyperparameter:', rf_tune_result_5.best_params_)

#### Compare Score

In [149]:
print('Best score 1st:', rf_tune_result_1.best_score_)
print('Best score 2nd:', rf_tune_result_2.best_score_)
print('Best score 3rd:', rf_tune_result_3.best_score_)
print('Best score 4th:', rf_tune_result_4.best_score_)
print('Best score 5th:', rf_tune_result_5.best_score_)

#### Conclusion
The hyperparameters at 4th iteration gives the best score, thus we will use it as our new model hyperparameters.

### 5.3.3. Model Comparison After Tuning

#### New Model

In [152]:
# define the model
new_rf_model = RandomForestClassifier(random_state=0, **rf_tune_result_4.best_params_)

# train the model
new_rf_result = new_rf_model.fit(X_train, y_train)

# prediction
new_rf_predicted = new_rf_result.predict(X_test)
new_rf_predicted_proba = new_rf_result.predict_proba(X_test)
new_rf_predicted_proba = [i[1] for i in new_rf_predicted_proba]

# confusion matrix
new_rf_conf_mat = confusion_matrix(y_true = y_test, y_pred = new_rf_predicted)
print("New Random Forest Confusion Matrix:")
print(new_rf_conf_mat)
print('\n\n')
# display confusion matrix heatmap
confusion_matrix_heatmap(new_rf_conf_mat)

print('\n\n')

# Evaluation
new_rf_accuracy = accuracy_score(y_true = y_test, y_pred = new_rf_predicted)
new_rf_fscore = f1_score(y_true = y_test, y_pred = new_rf_predicted)
new_rf_roc_auc = roc_auc_score(y_true = y_test, y_score = new_rf_predicted_proba)
print("New Random Forest Accuracy:", round(new_rf_accuracy,3))
print("New Random Forest F-Score:", round(new_rf_fscore, 3))
print("New Random Forest AUC:", round(new_rf_roc_auc, 3))
print('\n\n')
# display ROC curve
auc_visualization(y_test, new_rf_predicted_proba)

#### Score Comparison

In [154]:
# comparison
print("Random Forest Accuracy:", round(rf_accuracy,4))
print("Random Forest F-Score:", round(rf_fscore, 4))
print("Random Forest AUC:", round(rf_roc_auc, 4))
print('===')
print("New Random Forest Accuracy:", round(new_rf_accuracy,4))
print("New Random Forest F-Score:", round(new_rf_fscore, 4))
print("New Random Forest AUC:", round(new_rf_roc_auc, 4))

## 5.4. Conclusion

### Best Model
The model which has the best AUC score for our data is Random Forest, followed by Logistic Regression, and then k-NN in the last place.

In [155]:
print("New Logistic Regression AUC:", round(new_lg_roc_auc, 4))
print("New k-NN AUC:", round(new_knn_roc_auc, 4))
print("New Random Forest AUC:", round(new_rf_roc_auc, 4))

### Feature Importance

In [156]:
new_rf_model.feature_importances_

In [163]:
len(X.columns)

In [169]:
# feature importance dataframe
feat_imp = pd.DataFrame({'Feature': X.columns,
                         'Importance': new_rf_model.feature_importances_})
feat_imp_sort = feat_imp.sort_values(by='Importance', ascending=False)
feat_imp_sort

# feature importance bar plot


In [171]:
# display as figure
plt.figure(figsize=(7,5))
sns.barplot(x=feat_imp_sort['Importance'], y=feat_imp_sort['Feature'], color='#a2c9f4')
plt.ylabel('')
plt.xlabel('Importance')
plt.title('Feature Importance', fontsize=12)
plt.show()